# Natural Language Processing with Tensorflow

This notebook contains:
  * Downloading a text dataset
  * Visualizing the data
  * Converting text into numbers using tokenization
  * Turning tokenized text into an embedding
  * Modelling a text data
    * Starting with a baseline (TF-IDF)
    * Building several deep learning text models
      * Dense, LSTM, GRU, Conv1D, Transfer Learning
  * Comparing the performance of each of our models
  * Combining our models into an ensemble
  * Saving and loading a trained model
  * Finding the most wrong predictions

## Checking the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1262053a-e2f1-c944-8e3d-303652ece4e3)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-12 07:29:30--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-12 07:29:30 (80.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Download a text dataset

In [4]:
# Download data
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-02-12 07:29:33--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.128, 74.125.200.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   732KB/s    in 0.8s    

2023-02-12 07:29:35 (732 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip data
unzip_data("nlp_getting_started.zip")

# Visualizing the dataset

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many samples total
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df)+len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [11]:
# Visualizing some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: {text}")
  print("---")

Target: 1 (real disaster)
Text: 9000 Photographs from 1800's British Mandate of Palestine - with no trace of 'Palestinians' http://t.co/X8i0mHYRmN
---
Target: 0 (not real disaster)
Text: And you're loving me like water slipping through my fingers such a natural disaster love
---
Target: 0 (not real disaster)
Text: #stlouis #caraccidentlawyer Speeding Among Top Causes of Teen Accidents https://t.co/k4zoMOF319 https://t.co/S2kXVM0cBA Car Accident teeÛ_
---
Target: 1 (real disaster)
Text: Tram travellers evacuated after powerlines come down in Surfers http://t.co/Qsheu3yF0W
---
Target: 0 (not real disaster)
Text: @DanHRothschild Greed is the fuel of self-destruction. #Takecare
---


### Split the data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size=0.1, random_state=42)

In [13]:
# Check the sizes of train and val sets
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

# Converting Text into Numbers

### Text Vectorization

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [15]:
# Find average number of tokens in training Tweets
round(sum([len(i.split()) for i in X_train])/len(X_train))

15

In [16]:
#@title Default title text
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(X_train)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nVectorized text: \n{text_vectorizer(random_sentence)}")

Original text:
First wreck today. So so glad me and mom are okay. Could've been a lot worse. So happy the lord was with us today ??????
Vectorized text: 
[  97  184  124   28   28 1081   31    7  881   22  734 5908   59    3
  505]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

In [21]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length="max_length",
                             name="embedding_1")

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Meet Brinco your own personal earthquake snd tsunami early warning beacon. http://t.co/NXkUM9h7wD
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01968049, -0.04855068,  0.04038227, ..., -0.00848401,
          0.00588304,  0.03352313],
        [ 0.0448058 , -0.03800714,  0.0120662 , ...,  0.02699495,
          0.03200877,  0.01159061],
        [ 0.00251215,  0.01794795, -0.00101835, ...,  0.04601766,
          0.04355359,  0.01795227],
        ...,
        [ 0.04257135,  0.03924853,  0.04299835, ..., -0.0452073 ,
         -0.01668004, -0.0387238 ],
        [ 0.04257135,  0.03924853,  0.04299835, ..., -0.0452073 ,
         -0.01668004, -0.0387238 ],
        [ 0.04257135,  0.03924853,  0.04299835, ..., -0.0452073 ,
         -0.01668004, -0.0387238 ]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.01968049, -0.04855068,  0.04038227, -0.04728775, -0.00877066,
        0.03120932,  0.01955576,  0.0386622 , -0.03522797, -0.03080716,
       -0.03280431,  0.02496544,  0.02654976,  0.01093186, -0.03732311,
       -0.04369472,  0.04333404,  0.04065103, -0.02027476,  0.04292232,
       -0.04097396,  0.00800667, -0.01627783, -0.01962147, -0.00040609,
        0.00715365,  0.04308954, -0.0148658 , -0.04041626,  0.0279067 ,
        0.01053691,  0.04236606,  0.02002361, -0.02537937,  0.04061283,
       -0.01490051,  0.03643738,  0.02159139, -0.02735541,  0.04146287,
       -0.0459495 , -0.02722082, -0.00845234,  0.04282304,  0.00874778,
       -0.01202785, -0.04839026,  0.03856511,  0.01789765,  0.04386152,
        0.00971616, -0.04956543, -0.00617741,  0.01773696,  0.02830602,
        0.03153693, -0.03162103, -0.01320272,  0.03524036,  0.03339144,
       -0.00526248, -0.02567093, -0.04680857, -0.00735177, -0.00636474,
       -0.047701

# Modelling a text dataset

* **Model 0**: Naive bayes (baseline)
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: Tensorflow Hub Pretrained Feature Extractor
* **Model 7**: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(X_val, y_val)
print(f"Our baseline model achieves the accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves the accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments
  * Accuracy
  * Precision
  * Recall
  * F1-Score

In [27]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1":model_f1}
  return model_results

In [28]:
# Get baseline results
baseline_results = calculate_results(y_true=y_val, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A Simple dense model

In [29]:
# Create tensorboard callback
from helper_functions import create_tensorboard_callback

# Create directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [30]:
# Build model with the functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [31]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [32]:
# Get summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
# Fit the model
model_1.history = model_1.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=[X_val, y_val],
                              callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: SAVE_DIR/simple_dense_model/20230212-072940
Epoch 1/5
215/215 [==============================] - 24s 91ms/step - loss: 0.6101 - accuracy: 0.6891 - val_loss: 0.5364 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4412 - accuracy: 0.8202 - val_loss: 0.4694 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3462 - accuracy: 0.8622 - val_loss: 0.4594 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2846 - accuracy: 0.8932 - val_loss: 0.4644 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2377 - accuracy: 0.9115 - val_loss: 0.4768 - val_accuracy: 0.7874


In [34]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 3ms/step - loss: 0.4768 - accuracy: 0.7874


[0.4768275022506714, 0.787401556968689]

In [35]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.06050203,  0.05694026,  0.02822336, ..., -0.02927578,
          0.00058556, -0.02404113],
        [ 0.04768558, -0.0350699 ,  0.01343433, ...,  0.02865526,
          0.03456131,  0.01264466],
        [ 0.01343834,  0.00385637, -0.04741177, ...,  0.05491715,
          0.00775623, -0.02820847],
        ...,
        [-0.04534856, -0.04745988, -0.01391036, ..., -0.0053687 ,
         -0.03268316,  0.04419952],
        [ 0.08225781,  0.01258897, -0.06179034, ..., -0.00451746,
          0.01528536, -0.00878978],
        [ 0.07382408,  0.10949318, -0.08623162, ...,  0.04853433,
          0.0626061 ,  0.10129516]], dtype=float32)>]

In [36]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [37]:
# Make predictions
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.41937724],
       [0.7456102 ],
       [0.997832  ],
       [0.11221225],
       [0.10805737],
       [0.9370465 ],
       [0.91708946],
       [0.99288684],
       [0.9694462 ],
       [0.26387733]], dtype=float32)

In [38]:
# Turn prediction probabilities into single dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [39]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true = y_val, y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

### Model 2: LSTM

In [40]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(1, activation="sigmoid")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)


In [41]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [42]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [43]:
# Fit the model
model_2_history = model_2.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230212-073013
Epoch 1/5
215/215 [==============================] - 12s 38ms/step - loss: 0.5101 - accuracy: 0.7454 - val_loss: 0.4550 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3180 - accuracy: 0.8702 - val_loss: 0.5190 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2181 - accuracy: 0.9169 - val_loss: 0.5856 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1523 - accuracy: 0.9461 - val_loss: 0.6449 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1052 - accuracy: 0.9602 - val_loss: 0.8085 - val_accuracy: 0.7572


In [44]:
# Make prediction on the validation dataset
model_2_pred_probs = model_2.predict(X_val)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1), array([[0.01427672],
        [0.722282  ],
        [0.9995055 ],
        [0.07927838],
        [0.00572446],
        [0.9993674 ],
        [0.9592469 ],
        [0.9997085 ],
        [0.99961436],
        [0.19139485]], dtype=float32))

In [45]:
# Round up predictions and reduce it to 1D array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [46]:
# Calculate LSTM model results
model_2_results = calculate_results(y_val, model_2_preds)
model_2_results

{'accuracy': 75.7217847769029,
 'precision': 0.7581583214678428,
 'recall': 0.7572178477690289,
 'f1': 0.755217155129904}

### Model 3: GRU

In [47]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU")

In [48]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [49]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________________

In [50]:
# Fit the model
model_3_history = model_3.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230212-073033
Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.5296 - accuracy: 0.7244 - val_loss: 0.4535 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3202 - accuracy: 0.8675 - val_loss: 0.4874 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2173 - accuracy: 0.9187 - val_loss: 0.5598 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1542 - accuracy: 0.9482 - val_loss: 0.6235 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1169 - accuracy: 0.9591 - val_loss: 0.6046 - val_accuracy: 0.7769


In [51]:
# Make prediction on validation data
model_3_pred_probs = model_3.predict(X_val)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.29935202],
       [0.92000014],
       [0.9964858 ],
       [0.15339631],
       [0.01722257],
       [0.9880684 ],
       [0.82473254],
       [0.99566793],
       [0.9954194 ],
       [0.4158958 ]], dtype=float32)

In [52]:
# Round up the prediction and convert it to 1D array
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [53]:
# Calculate result
model_3_results = calculate_results(y_val, model_3_preds)
model_3_results

{'accuracy': 77.69028871391076,
 'precision': 0.7771036524564976,
 'recall': 0.7769028871391076,
 'f1': 0.7757784765658782}

### Model 4: Bidirectional RNN Model

In [54]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in tensorflow
input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
output = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(input, output, name="model_4_Bidirectional")

In [55]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [56]:
# Get a summary of our bidirectional model
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [57]:
# Fit the model
model_4_history = model_4.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20230212-073054
Epoch 1/5
215/215 [==============================] - 12s 39ms/step - loss: 0.5126 - accuracy: 0.7466 - val_loss: 0.4595 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3142 - accuracy: 0.8720 - val_loss: 0.5140 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2129 - accuracy: 0.9184 - val_loss: 0.5693 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1460 - accuracy: 0.9505 - val_loss: 0.6487 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1050 - accuracy: 0.9642 - val_loss: 0.6770 - val_accuracy: 0.7677


In [58]:
# Make predictions with bidirectional RNN on validation data
model_4_pred_probs = model_4.predict(X_val)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[0.09069788],
       [0.8076969 ],
       [0.9990947 ],
       [0.21339811],
       [0.00447003],
       [0.995993  ],
       [0.9704329 ],
       [0.99941957],
       [0.99928516],
       [0.12257694]], dtype=float32)

In [59]:
# Convert prediction probabilities to labels
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [60]:
# Calculate bidirection model results
model_4_results = calculate_results(y_val, model_4_pred)
model_4_results

{'accuracy': 76.77165354330708,
 'precision': 0.7678558301291274,
 'recall': 0.7677165354330708,
 'f1': 0.7664937875658134}

### Model 5: Conv1D (Convolutional Neural Network for Text)

In [61]:
# Test out the embedding, 1D Convolutional and max pooling
embedding_test = embedding(text_vectorizer(['This is a test sentence']))
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [62]:
embedding_test[:1], max_pool_output[:1], conv_1d_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.04587274,  0.03996259, -0.02524789, ...,  0.02271367,
          -0.00992495,  0.0318286 ],
         [ 0.05548596,  0.04754952, -0.0918293 , ...,  0.00437542,
           0.06916443, -0.00669172],
         [ 0.05105571, -0.00599601, -0.04768273, ..., -0.02769615,
           0.00822923,  0.01697678],
         ...,
         [ 0.06050203,  0.05694026,  0.02822336, ..., -0.02927578,
           0.00058556, -0.02404113],
         [ 0.06050203,  0.05694026,  0.02822336, ..., -0.02927578,
           0.00058556, -0.02404113],
         [ 0.06050203,  0.05694026,  0.02822336, ..., -0.02927578,
           0.00058556, -0.02404113]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 32), dtype=float32, numpy=
 array([[0.10028905, 0.08079337, 0.09268863, 0.06459353, 0.12214273,
         0.        , 0.01575203, 0.13089891, 0.10044755, 0.14582756,
         0.091682  , 0.01873143, 0.10670686, 0.        , 0.        ,
         0.11723603, 0.        

In [63]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [64]:
# model_5 summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1920      
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [65]:
# Compile model_5
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
# Fit the model
model_5_history = model_5.fit(X_train, y_train, epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230212-073122
Epoch 1/5
215/215 [==============================] - 12s 45ms/step - loss: 0.6402 - accuracy: 0.6269 - val_loss: 0.6211 - val_accuracy: 0.6457
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.6115 - accuracy: 0.6564 - val_loss: 0.6112 - val_accuracy: 0.6522
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.6022 - accuracy: 0.6681 - val_loss: 0.6118 - val_accuracy: 0.6535
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.5964 - accuracy: 0.6720 - val_loss: 0.6123 - val_accuracy: 0.6575
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.5897 - accuracy: 0.6795 - val_loss: 0.6140 - val_accuracy: 0.6601


In [67]:
# Make predictions with model_5
model_5_pred_probs = model_5.predict(X_val)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.4254837 ],
       [0.7753449 ],
       [0.7822852 ],
       [0.20099398],
       [0.4129646 ],
       [0.72114265],
       [0.74223775],
       [0.4529662 ],
       [0.6688565 ],
       [0.37442407]], dtype=float32)

In [68]:
# Convert predictions to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 0., 1., 0.], dtype=float32)>

In [69]:
# Calculate model_5 results
model_5_results = calculate_results(y_val, model_5_preds)
model_5_results

{'accuracy': 66.01049868766404,
 'precision': 0.6682312689422611,
 'recall': 0.6601049868766404,
 'f1': 0.6460108750666029}

### Model 6: TensorFlow Hub Pretrained Sentence Encoder

In [71]:
# Example pof pre trained embedding with universal sentence encoder
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed(["When you call the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])


tf.Tensor(
[ 0.0359669  -0.08579469 -0.01152741  0.00525985 -0.01852173 -0.05042012
 -0.03616942  0.00534677  0.04805915  0.04690744 -0.0372333  -0.01149543
  0.04352415  0.07050992  0.07093763 -0.08180429  0.00871717 -0.04654122
 -0.02245775  0.04686873  0.00202256  0.03099073  0.02043564  0.06392168
 -0.07641087  0.08421178 -0.04576039 -0.00106166 -0.0205942   0.01241105
  0.05727538  0.03815621 -0.02742114 -0.00354347 -0.09832585 -0.01244855
  0.03865628  0.05031954 -0.02362506  0.00321848  0.03225213  0.00738092
  0.04473102 -0.00412238  0.01151604  0.02837726  0.00060137 -0.05903351
 -0.00494344 -0.0068853 ], shape=(50,), dtype=float32)


In [72]:
embed_samples[0].shape

TensorShape([512])

In [76]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                         input_shape=[],
                                         dtype="string",
                                         trainable=False,
                                         name="USE")


In [78]:
# Create model using sequential API
model_6 = tf.keras.Sequential([
    sentence_encoding_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile the model
model_6.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["Accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [79]:
# Fit the model
model_6_history = model_6.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder"))

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230212-074944
Epoch 1/5
215/215 [==============================] - 10s 22ms/step - loss: 0.5060 - Accuracy: 0.7818 - val_loss: 0.4465 - val_Accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.4142 - Accuracy: 0.8148 - val_loss: 0.4353 - val_Accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3990 - Accuracy: 0.8221 - val_loss: 0.4316 - val_Accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3915 - Accuracy: 0.8272 - val_loss: 0.4276 - val_Accuracy: 0.8163
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.3851 - Accuracy: 0.8301 - val_loss: 0.4296 - val_Accuracy: 0.8150


In [80]:
# Make predictions
model_6_pred_probs = model_6.predict(X_val)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 12ms/step


array([[0.16539764],
       [0.74757797],
       [0.9893321 ],
       [0.21612191],
       [0.7163876 ],
       [0.69382083],
       [0.9820909 ],
       [0.9784418 ],
       [0.9257013 ],
       [0.08972691]], dtype=float32)

In [81]:
# Convert probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [82]:
# Calculate model_6 results
model_6_results = calculate_results(y_val, model_6_preds)
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8182618558292442,
 'recall': 0.8149606299212598,
 'f1': 0.8131230163581015}